In [1]:
import sys, os, random
import csv
import json
import openai
from datasets import load_dataset
from collections import defaultdict, OrderedDict
import shutil
import ast, re, copy
import uuid
from tqdm import tqdm
from mypy import api as mypyapi
import subprocess
import mistune

In [2]:
#temperature, top_p = 0.1, 1.0
temperature, top_p = 1.0, 0.7
#model = "gpt-4o-mini-2024-07-18"
#model = "gpt-4o-2024-05-13"
#model = "gpt-4-turbo-2024-04-09"
#model = "ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-0-round-0:AHFhDD6I"
#model = "ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-1-round-0:AIf2wVxd"
#model = "ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-2-round-0:AJ3BKMSO"
#model = "ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-0-round-1:AKsO2x9L"
#model = "ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-0-round-2:AKxe9TmB"
#model = "ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-0-round-3:Adfdi05r"
#model = "ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-0-round-4:AfXtDlh5"
#model = "ft:gpt-4o-2024-08-06:uedin:semantic-inequiv-alice-run-0-round-5-dpo:Aq5l34VS"
model = "ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-0-round-5:Aphr2tfR"
max_tokens = 2048
n = 10

In [3]:
dataset_full = load_dataset("mbpp")

In [4]:
system_message = \
"""You are an expert computer scientist. Your task is to take a Python 3.10 program and write a similar program which is not semantically equivalent, which means that there must exist at least a diverging input example such that the original program and your program either produce different outputs or exceptions, or one halts and the other one does not halt. In addition to a program, you need to produce a diverging input example. Start by carefully analyzing the original program and think of how an example would propagate through it from the input to the return value, considering how to modify the program in order to elicit a different behavior. Make sure that the return values or exceptions raised by your program are picklable.
The original program and your program will be used in a test to evaluate the skill of an expert computer scientist who will have to produce a diverging example (not necessarily the same as yours), so make sure that the difference you introduce are not very easy to understand. You will be given a difficulty level from 0 (easiest) to 10 (hardest) to target. E.g. difficulty level 0 means that an expert computer scientist in the bottom decile or above should be able to find a diverging example, difficulty level 9 means that only an expert computer scientist in the top decile should be able to find a diverging example, and difficulty level 10 means that only the top 1% or less of expert computer scientists should be able to find a diverging example.
Think step by step before writing your program. Use the following Markdown format, making sure that the following sections are delimited by level 1 headings, since they will have to be automatically parsed:
# Analysis
step by step analysis. This section can include sub-headings and code blocks
# Generated program
your program inside a Python code block. Do not change the name or signature of the entry point function
# Diverging input example
your diverging input example as a Python dictionary inside a Python code block
For instance, if the entry point function takes two parameters a and b and your diverging example is a="foo" and b=42, write:
```python
{
  "a": "foo",
  "b": 42
}
```
do not write the expected outputs
"""

def user_message_fn_0(example, difficulty_level=10):
    return f"""Difficulty level: {difficulty_level}
Entry point function: {example["function_name"]}

```python
{example["code"]}
```"""

In [5]:
def example_to_openai_api_format(example, difficulty_level=10):
    messages = [{"role": "system", "content": system_message}, 
                {"role": "user", "content": user_message_fn_0(example, difficulty_level)}]
    rv = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "top_p": top_p,
        "max_tokens": max_tokens,
        "n": n
#        "response_format": { "type": "json_object" }
    }
    return json.dumps(rv)

In [6]:
round_n = 6
run_n = 1
#is_test = False
is_test = True

train_or_test_str = "" if not is_test else "_test"
example_to_openai_file_name_base = f"./generations{train_or_test_str}_round_{round_n}_run_{run_n}_to_openai.jsonl"
example_from_openai_file_name_base = f"./generations{train_or_test_str}_round_{round_n}_run_{run_n}_from_openai.jsonl"

In [7]:
example_to_openai_file_name_base

'./generations_test_round_6_run_1_to_openai.jsonl'

In [8]:
data = dataset_full["train"] if not is_test else dataset_full["test"]
data = [dict(example) for example in data]
new_data = list()
for example in data:
    function_name = ast.parse(example["test_list"][0]).body[0].test.left.func.id
    example["function_name"] = function_name
    new_example = defaultdict(list)
    new_example.update(example)
    new_data.append(new_example)
data = new_data

In [9]:
data[0].keys()

dict_keys(['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list', 'function_name'])

In [10]:
data[0]['user_message']

[]

In [11]:
print(user_message_fn_0(data[0]))
print(example_to_openai_api_format(data[0]))

Difficulty level: 10
Entry point function: remove_Occ

```python
def remove_Occ(s,ch): 
    for i in range(len(s)): 
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    for i in range(len(s) - 1,-1,-1):  
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    return s 
```
{"model": "ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-0-round-5:Aphr2tfR", "messages": [{"role": "system", "content": "You are an expert computer scientist. Your task is to take a Python 3.10 program and write a similar program which is not semantically equivalent, which means that there must exist at least a diverging input example such that the original program and your program either produce different outputs or exceptions, or one halts and the other one does not halt. In addition to a program, you need to produce a diverging input example. Start by carefully analyzing the original program and think of how an example would propagate t

In [12]:
with open(example_to_openai_file_name_base, "w") as out_fs:
    for example in data:
        if len(example["test_correct_programs"]) > 0:
            # already done, skip
            continue
        print(example_to_openai_api_format(example), file=out_fs)

In [13]:
filename = example_from_openai_file_name_base
#filename = example_with_errors_from_openai_file_name_base

alice_round_0_generations_oai = []

response_dict = defaultdict(list)
with open(filename, "r") as in_fs:
    for i, line in enumerate(in_fs):
        response = json.loads(line)
        inner_input = response[0]["messages"][1]["content"]
        for j, inner_output in enumerate(response[1]["choices"]):
            finish_reason = inner_output["finish_reason"]
            if finish_reason == "stop":
                generate_valid = True
            else:
                print(f"{i}, {j} {inner_input}, {finish_reason}")
                print(inner_output)
                print("")
                continue
            response_dict[inner_input].append(inner_output)

# reorder
new_response_dict = OrderedDict()
with open(example_to_openai_file_name_base, "r") as in_fs:
    for line in in_fs:
        original = json.loads(line)
        inner_input = original["messages"][1]["content"]
        new_response_dict[inner_input] = response_dict[inner_input]
alice_round_0_generations_oai = list(new_response_dict.items())

345, 9 Difficulty level: 10
Entry point function: is_num_keith

```python
def is_num_keith(x): 
	terms = [] 
	temp = x 
	n = 0 
	while (temp > 0): 
		terms.append(temp % 10) 
		temp = int(temp / 10) 
		n+=1 
	terms.reverse() 
	next_term = 0 
	i = n 
	while (next_term < x): 
		next_term = 0 
		for j in range(1,n+1): 
			next_term += terms[i - j] 
		terms.append(next_term) 
		i+=1 
	return (next_term == x) 
```, length
{'index': 9, 'message': {'role': 'assistant', 'content': '# Analysis\nThe provided function `is_num_keith` checks if a number `x` is a Keith number. A Keith number is defined by a specific sequence generated from its digits. The function works as follows:\n\n1. **Extract Digits**: It extracts the digits of the number `x` and stores them in a list called `terms`.\n2. **Generate Sequence**: It generates a sequence of numbers starting from the sum of the digits, appending each new term to the list until the next term is equal to or greater than `x`.\n3. **Check Equality**: Fi

In [14]:
alice_round_0_generations_oai[0][0]

'Difficulty level: 10\nEntry point function: remove_Occ\n\n```python\ndef remove_Occ(s,ch): \r\n    for i in range(len(s)): \r\n        if (s[i] == ch): \r\n            s = s[0 : i] + s[i + 1:] \r\n            break\r\n    for i in range(len(s) - 1,-1,-1):  \r\n        if (s[i] == ch): \r\n            s = s[0 : i] + s[i + 1:] \r\n            break\r\n    return s \n```'

In [15]:
# Extract generated program and diverging examples

#python_code_block_pattern = r"```python(.*?)```"
markdown = mistune.create_markdown(renderer='ast')

lodaded_responses = dict(alice_round_0_generations_oai)
for i, example in enumerate(data):
    if len(example["test_correct_programs"]) > 0:
        # already done, skip
        continue
    user_message = user_message_fn_0(example)
    entry_v = lodaded_responses.get(user_message, None)
    if entry_v is None:
        continue
    example["user_message"] = user_message
    example["new_raw_programs_and_examples"] = []
    programs_and_examples_set = set()
    for k, response in enumerate(entry_v):
        content = response["message"]["content"]
        try:
            parsed_markdown = markdown(content)
            # Find "Generated program" and "Diverging input example"
            gp_i = [i for i, e in enumerate(parsed_markdown)  if (e['type'] == "heading") and (e["children"][0]["raw"] == "Generated program")][0]
            die_i = [i for i, e in enumerate(parsed_markdown) if (e['type'] == "heading") and (e["children"][0]["raw"] == "Diverging input example")][0]
            # Extract code blocks
            program_code_block = [e["raw"] for e in parsed_markdown[gp_i:die_i] if (e['type'] == "block_code") and ("attrs" in e) and (e["attrs"]["info"]=="python")][0]
            die_code_block     = [e["raw"] for e in parsed_markdown[die_i:]     if (e['type'] == "block_code") and ("attrs" in e) and (e["attrs"]["info"]=="python")][0]
        except:
            print(f"Error: {i}, {k} invalid markdown")
            #print(content)
            continue
        program_and_example_key = (("program", program_code_block), ("diverging_input_example", die_code_block))
        if program_and_example_key not in programs_and_examples_set:
            programs_and_examples_set.add(program_and_example_key)
            example["new_raw_programs_and_examples"].append({"program": program_code_block,
                                                             "diverging_input_example": die_code_block,
                                                             "id": k,
                                                             "llm_response": content})

In [159]:
# Alternatively, load precomputed

in_filename = f"./alice_train_round_{round-1}.jsonl"

data = list()
with open(in_filename, "r") as in_fs:
    for line in in_fs:
        example = json.loads(line.strip())
        new_example = defaultdict(list)
        new_example.update(example)
        new_example["bad_programs"] = []
        data.append(new_example)

In [16]:
data[0].keys()

dict_keys(['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list', 'function_name', 'user_message', 'test_correct_programs', 'new_raw_programs_and_examples'])

In [17]:
data[0]["raw_programs_and_examples"] == data[0]["new_raw_programs_and_examples"]

False

In [18]:
# Parse and unparse

for i, example in enumerate(data):
    if len(example["test_correct_programs"]) > 0:
        # already done, skip
        continue
    example["new_programs_and_examples"] = []
    for k, program_and_example in enumerate(example["new_raw_programs_and_examples"]):
        try:
            program, die = program_and_example["program"], program_and_example["diverging_input_example"]
            parsed_program = ast.parse(program, type_comments=True)
            parsed_die = ast.parse(die, type_comments=True)
            example["new_programs_and_examples"].append({"program": ast.unparse(parsed_program), 
                                                         "diverging_input_example": ast.unparse(parsed_die),
                                                         "id": program_and_example["id"],
                                                         "llm_response": program_and_example["llm_response"]})
            
        except SyntaxError as e:
            print(f"Error in program {i}, {k}")
            print(e)
            error_message = f"{e.text}\n{str(e)}"
            example["bad_programs"].append({"program": program, 
                                            "diverging_input_example": die, 
                                            "error": error_message, 
                                            "error_type": "ast", 
                                            "id": program_and_example["id"],
                                            "llm_response": program_and_example["llm_response"]})

Error in program 18, 1
closing parenthesis '}' does not match opening parenthesis '[' (<unknown>, line 14)
Error in program 134, 4
closing parenthesis '}' does not match opening parenthesis '[' (<unknown>, line 8)
Error in program 177, 0
invalid syntax (<unknown>, line 5)
Error in program 213, 3
invalid syntax (<unknown>, line 8)
Error in program 323, 0
invalid syntax (<unknown>, line 4)
Error in program 365, 1
cannot use assignment expressions with subscript (<unknown>, line 3)
Error in program 455, 3
invalid syntax (<unknown>, line 4)


In [19]:
tmp_program_dir = "./tmp_programs_eval/"

In [20]:
def generate_test_harness(example, k, base_module_name, test_entrypoint_name="test_main", timeout_min_seconds=2.5, timeout_max_seconds=5.0):
    function_name = example["function_name"]
    die = example["new_programs_and_examples"][k]["diverging_input_example"]
    original_module_name = base_module_name+"_a"
    alt_module_name = base_module_name+"_b"
    timeout_seconds = random.uniform(timeout_min_seconds, timeout_max_seconds)
    rv = f"""
import multiprocessing
import pickle

def run_function(func, result_queue):
    try:
        rv = func()
        _ = pickle.dumps(rv)
        result_queue.put(("returned", rv))
    except BaseException as e:
        result_queue.put(("exception", e))

def run_with_timeout(func, timeout):
    result_queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=run_function, args=(func, result_queue))
    process.start()
    process.join(timeout)
    if process.is_alive():
        process.terminate()
        return ("timeout", None)
    else:
        return result_queue.get()

import {original_module_name}
import {alt_module_name}

def main():
    timeout = {timeout_seconds}
    rv_a = run_with_timeout({original_module_name}.{test_entrypoint_name}, timeout)
    rv_b = run_with_timeout({alt_module_name}.{test_entrypoint_name}, timeout)
    assert(rv_a != rv_b)

if __name__ == "__main__":
    main()

    """
    return rv.strip()

def generate_test_module(program, function_name, diverging_input_example, test_entrypoint_name="test_main"):
    #diverging_input_example_str = f"{[k+'='+v for k, v in diverging_input_example.items()]}"
    rv = f"""
{program}

def {test_entrypoint_name}():
  return {function_name}(**{diverging_input_example})
"""
    return rv.strip()

In [21]:
print(generate_test_harness(data[42], 0, "module_name"))
print("")
print(generate_test_module(data[42]["code"], 
                           data[42]["function_name"], 
                           data[42]["new_programs_and_examples"][0]["diverging_input_example"]))
print("")
print(generate_test_module(data[42]["new_programs_and_examples"][0]["program"], 
                           data[42]["function_name"], 
                           data[42]["new_programs_and_examples"][0]["diverging_input_example"]))

import multiprocessing
import pickle

def run_function(func, result_queue):
    try:
        rv = func()
        _ = pickle.dumps(rv)
        result_queue.put(("returned", rv))
    except BaseException as e:
        result_queue.put(("exception", e))

def run_with_timeout(func, timeout):
    result_queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=run_function, args=(func, result_queue))
    process.start()
    process.join(timeout)
    if process.is_alive():
        process.terminate()
        return ("timeout", None)
    else:
        return result_queue.get()

import module_name_a
import module_name_b

def main():
    timeout = 4.623340802769355
    rv_a = run_with_timeout(module_name_a.test_main, timeout)
    rv_b = run_with_timeout(module_name_b.test_main, timeout)
    assert(rv_a != rv_b)

if __name__ == "__main__":
    main()

def check_Equality(str):
  if (str[0] == str[-1]):  
    return ("Equal") 
  else:  
    return ("Not Equal") 

def test_main()

In [22]:
# Full check

num_bad = 0
num_test_checks = 0
num_type_failures, num_test_failures = 0, 0
for i in tqdm(range(len(data))):
    example = data[i]
    if len(example["test_correct_programs"]) > 0:
        # already done, skip
        continue
    if len(example["new_programs_and_examples"]) == 0:
        num_bad += 1
        continue
    example["raw_programs_and_examples"].extend(example["new_raw_programs_and_examples"])
    example["programs_and_examples"].extend(example["new_programs_and_examples"])
    for k, program_and_example in enumerate(example["new_programs_and_examples"]):
        module_name_base = "tmp_"+uuid.uuid4().hex+f"_{k}_x"
        original_module_name = module_name_base+"_a"
        alt_module_name = module_name_base+"_b"
        filename_original = tmp_program_dir+original_module_name+".py"
        filename_alt = tmp_program_dir+alt_module_name+".py"
        filename_harness = tmp_program_dir+module_name_base+"_harness.py"
        test_entrypoint_name = "test_"+uuid.uuid4().hex+"_main"
        with open(filename_original, "w") as out_fs:
            original_program = generate_test_module(example["code"], 
                                                    example["function_name"], 
                                                    program_and_example["diverging_input_example"],
                                                    test_entrypoint_name)
            print(original_program, file=out_fs)
        with open(filename_alt, "w") as out_fs:
            alt_program = generate_test_module(program_and_example["program"], 
                                               example["function_name"], 
                                               program_and_example["diverging_input_example"],
                                               test_entrypoint_name)
            print(alt_program, file=out_fs)
        with open(filename_harness, "w") as out_fs:
            test_harness = generate_test_harness(example, k, module_name_base, test_entrypoint_name=test_entrypoint_name)
            print(test_harness, file=out_fs)
        # execute tests
        cmd = f"python -B {filename_harness} > {filename_harness}.out 2>&1"
        cmd_rv = os.system(cmd)
        num_test_checks += 1
        if cmd_rv != 0:
            with open(f"{filename_harness}.out", "r") as err_fs:
                program_output = err_fs.read()
                example["bad_programs"].append({"program": program_and_example["program"],
                                                "diverging_input_example": program_and_example["diverging_input_example"],
                                                "error": program_output.strip(), 
                                                "error_type": "test",
                                                "id": program_and_example["id"],
                                                "llm_response": program_and_example["llm_response"]})
                num_test_failures += 1
        else:
            example["test_correct_programs"].append(program_and_example)
        os.remove(f"{filename_harness}.out")
        os.remove(filename_harness)
        os.remove(filename_alt)
        os.remove(filename_original)
    if len(example["test_correct_programs"]) == 0:
        num_bad += 1
print(f"Bad examples: {num_bad}/{len(data)}")
print(f"Test failures/checks: {num_test_failures}/{num_test_checks}")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [10:17<00:00,  1.24s/it]

Bad examples: 1/500
Test failures/checks: 931/4739


In [23]:
# Round 0 gpt-4o-mini-2024-07-18, temperature = 1.0, top_p = 0.7, n = 10, Bad examples: 1/374, Test failures/checks: 765/3692
# Round 0 (alt) gpt-4o-mini-2024-07-18, temperature = 1.0, top_p = 0.7, n = 10, Bad examples: 2/374, Test failures/checks: 734/3697
# Round 1 run 0 ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-0-round-0:AHFhDD6I, temperature = 1.0, top_p = 0.7, n = 10, Bad examples: 1/374, Test failures/checks: 699/3619
# Round 1 run 1 ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-1-round-0:AIf2wVxd, temperature = 1.0, top_p = 0.7, n = 10, Bad examples: 1/374, Test failures/checks: 693/3596
# Round 1 run 2 ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-2-round-0:AJ3BKMSO, temperature = 1.0, top_p = 0.7, n = 10, Bad examples: 1/374, Test failures/checks: 696/3588
# Round 2 run 0 ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-0-round-1:AKsO2x9L, temperature = 1.0, top_p = 0.7, n = 10, Bad examples: 0/374, Test failures/checks: 500/2975
# Round 3 run 0 ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-0-round-2:AKxe9TmB, temperature = 1.0, top_p = 0.7, n = 10, Bad examples: 0/374, Test failures/checks: 363/2301
# Round 4 run 0 ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-0-round-3:Adfdi05r, temperature = 1.0, top_p = 0.7, n = 10, Bad examples: 1/374, Test failures/checks: 538/3219
# Round 5 run 0 ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-0-round-4:AfXtDlh5, temperature = 1.0, top_p = 0.7, n = 10, Bad examples: 2/374, Test failures/checks: 548/3271
# Round 6 run 0 ft:gpt-4o-2024-08-06:uedin:semantic-inequiv-alice-run-0-round-5-dpo:Aq5l34VS, temperature = 1.0, top_p = 0.7, n = 10, Bad examples: 30/374, Test failures/checks: 1494/3078
# Round 6 run 1 ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-0-round-5:Aphr2tfR, temperature = 1.0, top_p = 0.7, n = 10, Bad examples: 1/374, Test failures/checks: 479/3052
# Test set Round 6 run 1 ft:gpt-4o-mini-2024-07-18:uedin:semantic-inequiv-alice-run-0-round-5:Aphr2tfR, temperature = 1.0, top_p = 0.7, n = 10, Bad examples: 1/500, Test failures/checks: 931/4739

In [25]:
(500-1)/500.0

0.998

In [26]:
4739-931

3808

In [27]:
[i for i, example in enumerate(data) if len(example["test_correct_programs"]) == 0]

[214]

In [28]:
train_or_test_str = "_train" if not is_test else "_test"

In [29]:
out_filename = f"./alice{train_or_test_str}_round_{round_n}_{run_n}.jsonl"

In [30]:
print(out_filename)

./alice_test_round_6_1.jsonl


In [31]:
with open(out_filename, "w") as out_fs:
    for example in data:
        print(json.dumps(example), file=out_fs)